In [1]:
import requests
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
import json
from delta import DeltaTable

spark = (SparkSession.builder
             .appName('lab') # Name the app
             .config("hive.metastore.uris", "thrift://metastore:9083") # Set external Hive Metastore
             .config("hive.metastore.schema.verification", "false") # Prevent some errors
             .config("spark.sql.repl.eagerEval.enabled", True)
             .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
             .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
             .enableHiveSupport()
             .getOrCreate())

dt_now = "2024-09-24"

In [3]:
spark.sql("show tables from business").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
| business|    posicao|      false|
| business|   previsao|      false|
| business|    paradas|      false|
| business|paradas_api|      false|
| business|      trips|      false|
| business| linhas_api|      false|
+---------+-----------+-----------+



In [6]:
spark.table("business.posicao").show(truncate=False)

+-----+-------+---+-------------------+----------------+---+---+----+-----+----------+----------+--------------------+
|Hora |c      |cl |lt0                |lt1             |qv |sl |a   |p    |px        |py        |ta                  |
+-----+-------+---+-------------------+----------------+---+---+----+-----+----------+----------+--------------------+
|23:00|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAPELINHA |9  |1  |true|71135|-46.69148 |-23.586823|2024-09-24T02:00:00Z|
|22:30|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAPELINHA |10 |1  |true|71160|-46.74717 |-23.647203|2024-09-24T01:29:37Z|
|23:00|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAPELINHA |9  |1  |true|71168|-46.752617|-23.649199|2024-09-24T02:00:02Z|
|22:37|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAPELINHA |9  |1  |true|71172|-46.63611 |-23.552088|2024-09-24T01:36:58Z|
|22:30|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAPELINHA |10 |1  |true|71181|-46.63749 |-23.552498|2024-09-24T01:30:06Z|
|23:00|5119-10|1  |LGO. SÃO FRANCISCO |TERM. CAP